In [ ]:
import pickle
import matplotlib.pyplot as plt
%matplotlib inline

import networkx as nx
import pandas as pd
import numpy as np

Run the following command within a CMSSW-aware environment to load the ROOT ntuple produced using `Validation/RecoParticleFlow/plugins/PFAnalysis.cc` and produce python-friendly data files:

```bash
python test/postprocessing2.py --input data/TTbar_14TeV_TuneCUETP8M1_cfi/pfntuple_1.root \
  --events-per-file 1 \
  --save-full-graph \
  --save-normalized-table \
  --save-images
```

The output will be stored in `data/TTbar_14TeV_TuneCUETP8M1_cfi/raw/pfntuple_1_*.pkl`.

In [ ]:
!wget http://login-1.hep.caltech.edu/~jpata/particleflow/2020-04/TTbar_14TeV_TuneCUETP8M1_cfi/pickle/pfntuple_1_0.pkl

In [ ]:
all_data = pickle.load(open("pfntuple_1_0.pkl", "rb"), encoding='iso-8859-1')

In [ ]:
one_event = all_data[0]
one_event.keys()

In [ ]:
graph = one_event["full_graph"]

In [ ]:
graph

In [ ]:
all_elements = [n for n in graph.nodes if n[0] == "elem"]
len(all_elements)

In [ ]:
all_gen = [n for n in graph.nodes if n[0] == "tp" or n[0] == "sc"]
len(all_gen)

In [ ]:
all_pfcand = [n for n in graph.nodes if n[0] == "pfcand"]
len(all_pfcand)

In [ ]:
def draw_event(g):
    pos = {}
    for node in g.nodes:
        pos[node] = (g.nodes[node]["eta"], g.nodes[node]["phi"])

    fig = plt.figure(figsize=(5,5))

    nodes_to_draw = [n for n in g.nodes if n[0]=="elem"]
    nx.draw_networkx(g, pos=pos, with_labels=False, node_size=5, nodelist=nodes_to_draw, edgelist=[], node_color="red", node_shape="s", alpha=0.5)

    nodes_to_draw = [n for n in g.nodes if n[0]=="pfcand"]
    nx.draw_networkx(g, pos=pos, with_labels=False, node_size=10, nodelist=nodes_to_draw, edgelist=[], node_color="green", node_shape="x", alpha=0.5)

    nodes_to_draw = [n for n in g.nodes if (n[0]=="sc" or n[0]=="tp")]
    nx.draw_networkx(g, pos=pos, with_labels=False, node_size=1, nodelist=nodes_to_draw, edgelist=[], node_color="blue", node_shape=".", alpha=0.5)

    #draw edges between genparticles and elements
    edges_to_draw = [e for e in g.edges if e[0] in nodes_to_draw]
    nx.draw_networkx_edges(g, pos, edgelist=edges_to_draw, arrows=False, alpha=0.1)

    plt.xlim(-6,6)
    plt.ylim(-4,4)
    plt.tight_layout()
    plt.axis("on")
    plt.xlabel("$\eta$")
    plt.xlabel("$\phi$")
    return fig

In [ ]:
draw_event(one_event["full_graph"]);

## Normalized table

In [ ]:
df = pd.DataFrame(np.concatenate([one_event["Xelem"], one_event["ygen"], one_event["ycand"]], axis=-1))

In [ ]:
#show some of the more important columns
df[[0,2,3,4,13,15,16,17,21,23,24,25]]

In [ ]:
plt.figure(figsize=(10,10))
#Element to element input distance matrix - graph adjacency
plt.imshow(one_event["dm"].todense()>0, cmap="Greys", interpolation="none")

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(one_event["dm_elem_cand"].todense()>0, cmap="Greys", interpolation="none")

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(one_event["dm_elem_gen"].todense()>0, cmap="Greys", interpolation="none")